In [1]:
import numpy as np
import pandas as pd
import datetime
import random
import sklearn

import seaborn as sns
import matplotlib.pyplot as plt
import datetime

# Ignore useless warnings 
import warnings
warnings.filterwarnings(action="ignore")                 

In [2]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import StandardScaler
import sklearn.metrics as sm
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import mean_squared_error
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import VotingRegressor
from sklearn.svm import LinearSVR
from sklearn.svm import SVR
from prettytable import PrettyTable

In [52]:
df = pd.read_csv('processed.csv', index_col = 0)
df.head(20)

,Operational Demand (MW),Transpiration,Rain,Evaporation,Max Temp,Min Temp,Max Hum,Min Hum,Av Wind,Solar Rad,...,sin_month,cos_month,sin_Day_of_week,cos_Day_of_week,sin_Day_of_month,cos_Day_of_month,sin_Day_of_year,cos_Day_of_year,sin_week_of_year,cos_week_of_year
Date,,,,,,,,,,,,,,,,,,,,,
1/01/2009,95141.184,11.2,0,11.6,37.2,18.8,65,20,6.1,34.01,...,0.5,0.866025,0.433884,-0.900969,2.079117e-01,0.978148,0.017213,0.999852,0.120537,0.992709
2/01/2009,108058.504,10.2,0,12.8,36.1,21.9,70,25,5.31,34.25,...,0.5,0.866025,-0.433884,-0.900969,4.067366e-01,0.913545,0.034422,0.999407,0.120537,0.992709
3/01/2009,95788.860,9.4,0,11.8,34.4,20.3,79,29,5.45,34.11,...,0.5,0.866025,-0.974928,-0.222521,5.877853e-01,0.809017,0.051620,0.998667,0.120537,0.992709
4/01/2009,93777.404,10.5,0,11.2,36.5,18.5,62,20,5.25,34.05,...,0.5,0.866025,-0.781831,0.623490,7.431448e-01,0.669131,0.068802,0.997630,0.120537,0.992709
5/01/2009,112678.424,11.9,0,11.4,37.2,21.8,63,17,6.77,34.18,...,0.5,0.866025,0.000000,1.000000,8.660254e-01,0.500000,0.085965,0.996298,0.239316,0.970942
6/01/2009,111824.976,10.1,0,12,36.4,19.7,73,26,5.42,33.82,...,0.5,0.866025,0.781831,0.623490,9.510565e-01,0.309017,0.103102,0.994671,0.239316,0.970942
7/01/2009,110104.118,10.7,0,12,36.3,18.9,63,21,5.83,33.85,...,0.5,0.866025,0.974928,-0.222521,9.945219e-01,0.104528,0.120208,0.992749,0.239316,0.970942
8/01/2009,114229.828,10.5,0,10.8,37.2,20.3,60,24,5.19,33.96,...,0.5,0.866025,0.433884,-0.900969,9.945219e-01,-0.104528,0.137279,0.990532,0.239316,0.970942
9/01/2009,113488.238,10.3,0,12.6,37.1,19.1,67,27,5.28,34.4,...,0.5,0.866025,-0.433884,-0.900969,9.510565e-01,-0.309017,0.154309,0.988023,0.239316,0.970942


In [53]:
numeric_dtypes = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
numeric = []
for i in df.columns:
    if df[i].dtype not in numeric_dtypes:
        df[i]=df[i].str.replace(" ",'') # Replace all ' ' into none
        df[i]=pd.to_numeric(df[i])


In [92]:
df.index = pd.to_datetime(df.index,format= '%d/%m/%Y')
select_col=['Operational Demand (MW)', 'Transpiration', 'Rain', 'Evaporation',
       'Max Temp', 'Min Temp', 'Max Hum', 'Min Hum', 'Av Wind', 'Solar Rad',
       'state_holiday', 'sin_month', 'cos_month', 'sin_Day_of_week',
       'cos_Day_of_week', 'sin_Day_of_month', 'cos_Day_of_month',
       'sin_Day_of_year', 'cos_Day_of_year', 'sin_week_of_year',
       'cos_week_of_year']
y_col=['Operational Demand (MW)']
x_col=list(set(select_col)-set(y_col))

model_df=df[select_col].fillna(method='ffill',axis=0)
test_dat = model_df[(model_df.index >= '2021-12-01') & ('2022-04-01' > model_df.index)]
training_dat = model_df[(model_df.index < '2021-12-01') | ('2022-04-01' < model_df.index)]

In [93]:
X_train = training_dat.drop('Operational Demand (MW)',axis=1)
y_train = training_dat['Operational Demand (MW)']

X_test = test_dat.drop('Operational Demand (MW)', axis = 1) 
y_test = test_dat['Operational Demand (MW)']

In [94]:
def ImportanceDisplay(model):
    if 'grid' in model:
        impo_lst=[model,'.best_estimator_.feature_importances_']
    else:
        impo_lst=[model,'.feature_importances_']
    impo_str=''
    importance = eval(impo_str.join(impo_lst))
    #importance = rf_reg.feature_importances_
    table = PrettyTable(['Factors','Score'])
    for i,v in enumerate(importance):
	    table.add_row([x_col[i],round(v,4)])
    print(table)

In [95]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 4970 entries, 2009-01-01 to 2022-08-15
Data columns (total 21 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Operational Demand (MW)  4970 non-null   float64
 1   Transpiration            4928 non-null   float64
 2   Rain                     4964 non-null   float64
 3   Evaporation              4956 non-null   float64
 4   Max Temp                 4969 non-null   float64
 5   Min Temp                 4968 non-null   float64
 6   Max Hum                  4969 non-null   float64
 7   Min Hum                  4969 non-null   float64
 8   Av Wind                  4969 non-null   float64
 9   Solar Rad                4932 non-null   float64
 10  state_holiday            4970 non-null   int64  
 11  sin_month                4970 non-null   float64
 12  cos_month                4970 non-null   float64
 13  sin_Day_of_week          4970 non-null   float64
 14  cos_Da

In [97]:
forest = RandomForestRegressor(criterion='mse', 
                                max_depth=15, # depth of tree 
                                min_samples_leaf=100, 
                                random_state=42)

forest.fit(X_train, y_train)
y_predict_forest = forest.predict(X_test)
ny_test = len(y_test)
mape = (sum(np.abs((y_test - y_predict_forest)/y_test))/ny_test)*100
print('training mape', mape)

ImportanceDisplay('forest')
# print(forest.best_estimator_)
# print(forest.best_score_)


training mape 6.600976137903699
+------------------+--------+
|     Factors      | Score  |
+------------------+--------+
|  state_holiday   | 0.0025 |
|       Rain       | 0.0002 |
|  Transpiration   | 0.0005 |
| sin_Day_of_year  | 0.4312 |
|     Max Hum      | 0.2594 |
|    Solar Rad     | 0.0008 |
|   Evaporation    | 0.0001 |
|     Av Wind      | 0.0001 |
|     Min Temp     | 0.0503 |
| cos_Day_of_week  |  0.0   |
|     Min Hum      | 0.0001 |
|    sin_month     | 0.0002 |
| cos_Day_of_year  | 0.1744 |
|     Max Temp     | 0.0002 |
| sin_Day_of_week  | 0.0001 |
| sin_Day_of_month | 0.0001 |
| sin_week_of_year | 0.0024 |
| cos_week_of_year | 0.0759 |
| cos_Day_of_month | 0.0012 |
|    cos_month     | 0.0003 |
+------------------+--------+


In [100]:

select_col_fs=['Operational Demand (MW)','sin_Day_of_year','Max Hum','cos_Day_of_year']
y_col_fs=['Operational Demand (MW)']
x_col_fs=list(set(select_col)-set(y_col))
model_df_fs=df[select_col].fillna(method='ffill',axis=0)
test_dat_fs = model_df[(model_df.index >= '2021-12-01') & ('2022-04-01' > model_df.index)]
training_dat_fs = model_df[(model_df.index < '2021-12-01') | ('2022-04-01' < model_df.index)]
X_train_fs = training_dat_fs.drop('Operational Demand (MW)',axis=1)
y_train_fs = training_dat_fs['Operational Demand (MW)']

X_test_fs = test_dat_fs.drop('Operational Demand (MW)', axis = 1) 
y_test_fs = test_dat_fs['Operational Demand (MW)']
forest = RandomForestRegressor(criterion='mse', 
                                max_depth=15, # depth of tree 
                                min_samples_leaf=100, 
                                random_state=42)
forest.fit(X_train_fs, y_train_fs)
y_predict_forest_fs = forest.predict(X_test_fs)
ny_test_fs = len(y_test_fs)
mape_fs = (sum(np.abs((y_test_fs - y_predict_forest_fs)/y_test_fs))/ny_test_fs)*100
print('training mape', mape)

training mape 11.7382191867269
